# Setup

## Imports

In [15]:
# General
import numpy as np
import pandas as pd

# Preprocessing
from gensim.models import Word2Vec

In [16]:
import os
import sys
# Add the path to the utils folder
sys.path.append(os.path.abspath('../..'))
import importlib
# Custom modules
from utils import memory_usage
from config import run_config, MODELS_PATH, FEATURES_PATH, PROCESSED_DATA_PATH
importlib.reload(sys.modules['utils'])
importlib.reload(sys.modules['config'])

<module 'config' from 'e:\\College\\4- Senior 2\\Semester 1\\NLP\\Project\\config.py'>

## Config

In [17]:
run_config()

In [18]:
# from IPython.utils import io

# with io.capture_output() as captured:
#     %run ../../1-EDA_and_Preprocessing/Preprocessing/Preprocessing.ipynb

In [19]:
# df_train = pd.read_parquet(PROCESSED_DATA_PATH + '/train_preprocessing.parquet')
X_train = np.load(PROCESSED_DATA_PATH + '/X_train.npy', allow_pickle=True)
X_train[:5]

array([list(['three', 'pizzas', 'no', 'american', 'cheese', 'and', 'a', 'water', 'and', 'one', 'ginger', 'ale', 'and', 'a', 'san', 'pellegrino', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']),
       list(['three', 'large', 'pizzas', 'with', 'balsamic', 'glaze', 'and', 'three', 'party', 'sized', 'pies', 'with', 'just', 'a', 'little', 'cherry', 'tomato', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']),
       list(["i'd", 'like', 'a', 'lunch', 'sized', 'pizza', 'with', 'parsley', 'fried', 'onions', 'and', 'parmesan', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']),
       list(['two', 'regular', 'pizzas', 'without', 'any', 'caramelized', 'onions', '<PAD>', '<PAD>', '<PAD>',

# Word2Vec

[Add padding & <UNK> to word2vec model](https://gist.github.com/timotta/ceaec141cbb0b18740b5c48071d989ae)

In [35]:
model_name = "/word2vec_model.bin"
update_model = False
if update_model or not os.path.exists(MODELS_PATH + model_name):
    print(f"Creating '{model_name}'...")
    # Create a Word2Vec model
    EMBED_SIZE = 1024
    word2vec_model = Word2Vec(sentences=X_train, vector_size=EMBED_SIZE, window=5, min_count=1, workers=4)
    word2vec_model.wv.add_vector("<UNK>", np.zeros(EMBED_SIZE))
    word2vec_model.wv["<PAD>"] = np.zeros(EMBED_SIZE)

    # Save the trained model
    print(f"Saving '{model_name}'...")
    word2vec_model.save(MODELS_PATH + model_name)
else:
    print(f"Loading '{model_name}'...")
    # Load the trained model
    word2vec_model = Word2Vec.load(MODELS_PATH + model_name)
    
word2vec_model

Loading '/word2vec_model.bin'...


In [36]:
def load_word2vec_embeddings(model_name: str):
    print(f"Loading '{model_name}'...")
    word2vec_model = Word2Vec.load(MODELS_PATH + model_name)
    return word2vec_model

def get_word_embedding(word2vec_model, word):
    try:
        return word2vec_model.wv[word]
    except:
        return word2vec_model.wv["<UNK>"]
    
def get_sentence_embedding(word2vec_model, sentence):
    return np.array([get_word_embedding(word2vec_model, word) for word in sentence])

In [37]:
word2vec_model = load_word2vec_embeddings("/word2vec_model.bin")
get_sentence_embedding(word2vec_model, X_train[0])

Loading '/word2vec_model.bin'...


array([[-0.7740219 ,  0.347325  ,  0.7386268 , ...,  0.96870726,
        -0.08133107,  0.8631404 ],
       [-0.35578468, -0.11080138,  1.3113868 , ...,  0.84098065,
        -0.53606856,  0.04742647],
       [ 0.6323756 ,  0.2043454 ,  0.6946837 , ...,  0.00133738,
         0.3960193 , -0.65608805],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [ ]:
# X_train_embeddings = np.array([get_sentence_embedding(word2vec_model, sentence) for sentence in X_train])

In [3]:
memory_usage()

118.796875